# Activity
Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [22]:
import os
import io
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [23]:
# Funcoes para ler os emails
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            filepath = os.path.join(root, filename)
            inBody = False
            lines = []
            f = io.open(filepath, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield filepath, message

def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)
    return pd.DataFrame(rows, index=index)

In [24]:
# Carrega os dados
data = pd.concat([
    dataFrameFromDirectory('/home/Ront/Documentos/Card-13/emails/spam', 'spam'),
    dataFrameFromDirectory('/home/Ront/Documentos/Card-13/emails/ham', 'ham')
], ignore_index=True)

In [25]:
# Prepara os dados
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['message'].values)
y = data['class'].values

In [26]:
# Criando o vetor de caracteristicas a partir das mensagens
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['message'].values)  # Transforma as mensagens em uma matriz de contagem de palavras

# Definindo as classes (spam ou ham) como alvo
y = data['class'].values

# Dividindo os dados em conjunto de treino e conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criando o classificador Multinomial Naive Bayes
classifier = MultinomialNB()

# Treinando o classificador com os dados de treino
classifier.fit(X_train, y_train)

# Fazendo previsoes com os dados de teste
y_pred = classifier.predict(X_test)

# Avaliando a acuracia do modelo
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy do modelo: {accuracy:.4f}')
print(f'Total: {len(data)} emails | Treino: {X_train.shape[0]} | Teste: {X_test.shape[0]}\n')

Accuracy do modelo: 0.9667
Total: 3000 emails | Treino: 2100 | Teste: 900



In [ ]:
print('='*60)
print('TESTANDO EMAILS CUSTOMIZADOS:')
print('='*60 + '\n')

# Lista de emails para testar
emails_teste = [
    'DINHEIRO GRATIS! Clique aqui para ganhar R$1.000.000 AGORA!!!',  # spam obvio
    'Oi, podemos marcar uma reuniao para amanha as 15h?',  # ham normal
    'Compre viagra barato online. Melhores precos!',  # spam obvio
    'Obrigado pela ajuda com o projeto. Te vejo segunda.',  # ham normal
    'Voce foi selecionado para uma oferta especial',  # caso ambiguo
]

# Testando cada email
for i, email in enumerate(emails_teste, 1):
    email_vectorizado = vectorizer.transform([email])  # transforma o email em vetor
    predicao = classifier.predict(email_vectorizado)[0]  # faz a predicao
    
    print(f'{i}. Email: "{email[:50]}..."')
    print(f'   Predicao: {predicao.upper()}\n')


In [18]:
print('='*60)
print('TESTE SEU PROPRIO EMAIL:')
print('='*60 + '\n')

# Substitua pelo seu email de teste
meu_email = 'Write your test email here'

# Fazendo a predicao
meu_email_vectorizado = vectorizer.transform([meu_email])
minha_predicao = classifier.predict(meu_email_vectorizado)[0]

print(f'Seu email: "{meu_email}"')
print(f'Classificacao: {minha_predicao.upper()}')

TESTE SEU PROPRIO EMAIL:

Seu email: "Write your test email here"
Classificacao: HAM
